In [1]:
#Author: Hana Jaafari
#Date: April 28, 2020
#The objective of this script is to locally set-up the files for a protein family's parent protein-pseudogene pairs
#to run AWSEM simulations on Nots.

import sys
import requests
import os
import subprocess
from shutil import copyfile
from datetime import datetime

In [2]:
#These are filepaths for my local desktop.
root_directory="/Users/hanajaafari/Desktop"
hmm_script_repository=os.path.join(root_directory,"hmmer-3.2.1/src/hmmer-3.2.1/src")
ncbi_blast_repository=os.path.join(root_directory,"ncbi-blast-2.10.0+/bin")

##These are filepaths for Nots.
#root_directory="/scratch/hkj1"
#hmm_script_repository=os.path.join(root_directory,"hmmer/hmmer-3.3/src")
#ncbi_blast_repository="/opt/apps/software/MPI/GCC/8.3.0/OpenMPI/3.1.4/BLAST+/2.9.0/bin"

time_devolution_directory=os.path.join(root_directory,"Time-Devolution-Calc")
openawsem_repository=os.path.join(root_directory,"openawsem")
past_Nots_files=os.path.join(time_devolution_directory,"DCA_Energy_Calculations/11_09_2019_Nots_Pseudogene_Analysis")
family_hmm_repository=os.path.join(past_Nots_files,"11_09_2019_Nots_hmm_files")
family_pseudogene_aligned_files_repository= os.path.join(past_Nots_files,"11_27_2019_family_aligned_pseudogenes")
family_parentprotein_aligned_files_repository= os.path.join(past_Nots_files,"11_09_2019_Nots_ParentProtein_MSA")
family_parentprotein_pseudogene_sequences_repository=os.path.join(past_Nots_files,"11_27_2019_renamed_pseudogene_parentprotein_sequence_fasta")
parentsequences_DCAEnergy_files_respository=os.path.join(past_Nots_files,"11_09_2019_Nots_ParentGene_Energies")

#This is the directory collecting all of the families' results from this run.
families_analysis_repository=os.path.join(time_devolution_directory,f"AWSEM_Energy_Calculations/\
{datetime.today().strftime('%m_%d_%Y')}_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates")

try:
    os.mkdir(families_analysis_repository)
except:
    pass                                      

In [3]:
#This is a modification of an excerpt from the openawsem script '/Users/hanajaafari/Desktop/openawsem-master/helperFunctions/MultCha_prepFrags_index.py' 
#that blasts the pdb sequence. This function will use psiblast to find the known full structure of the given 
#parent protein sequence if it exists.
def blast_pdb(protein_identifier,protein_sequence,open_awsem,psiblast_repository):
    if not os.path.exists(f"./{protein_identifier}_psiblast_identified_structures.txt"):
        print("#####")
        print(f"Blasting {protein_identifier} to Find Its Complete Structures")
        print("#####")
        print("The protein sequence is", protein_sequence)      
        with open(f"./{protein_identifier}_sequence.fasta","w") as output:
            output.write(">{}\n{}\n".format(protein_identifier,protein_sequence))
            
        pdb_ID_log=[]    
        database=os.path.join(open_awsem,"pdbDB")
        # blast the whole sequence to identify homologs Evalue 0.005
        exeline = f"{psiblast_repository}/psiblast -num_iterations 1 -word_size 3 -evalue 0.005"
        exeline += " -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db " + \
            database + f" -query ./{protein_identifier}_sequence.fasta"
        print("finding homologs")
        print("executing::: " + exeline)
        homoOut = os.popen(exeline).read()
        homoOut = homoOut.splitlines()  # now an array
        for line in homoOut:
            entries = line.split()
            print(f"homologs: {entries}")
            if len(entries):
                with open(f"./{protein_identifier}_psiblast_identified_structures.txt","a") as f:
                    f.write("{}\n".format(entries))
                pdb_ID_log.append([entries[0],float(entries[5]),float(entries[6])])
    else:
        pdb_ID_log=[]
        with open(f"./{protein_identifier}_psiblast_identified_structures.txt","r") as f:
            for line in f:
                line=line.replace("'","").replace(",","").replace("[","").replace("]","")
                pdb_ID_log.append([line.split()[0],float(line.split()[5]),float(line.split()[6])])
    print(f"The homologs for {protein_identifier} are: {pdb_ID_log}")
    return pdb_ID_log

In [4]:
#This function extracts the PDB crystal structures that meet or exceed the percentage identity to a given parent 
#protein sequence.If there is not a crystal structure for the parent protein with 100% sequence identity, 
#then the parent protein structure will be predicted using AWSEM.
def find_best_pdb_for_sequence(protein_identifier,blast_results,pdb_database,open_awsem):
    import pypdb as py
    import urllib
    qualified_pdb=[]
    if not os.path.exists(f"./{protein_identifier}_psiblast_identified_x-ray_diffraction_structure.txt"):
        print("#####")
        print(f"Finding Qualifying Structure for {protein_identifier}")
        print("#####")
        if not blast_results:
            return
        qualified_blast_results=[[i,j,k] for [i,j,k] in blast_results if j == 100.0 and k==100.0]
        if not qualified_blast_results:
            print("None of the blast hits are exact matches!")
            return
        #We now loop over to find entries whose crystal structure is determined through X-ray diffraction. We try to find 
        #an entry with a crystal structure. Once these conditions are met, we break from the loop below.
        for [pdb_result,pdb_result_percent_identity,pdb_result_sequence_coverage] in qualified_blast_results:
            pdb_information=py.describe_pdb(pdb_result[:4])
            if "X-RAY DIFFRACTION" in pdb_information.values():
                try:
                    if not pdb_result[:4] in open(f"{open_awsem}/notExistPDBsList").read():
                        urllib.request.urlretrieve(f"https://files.rcsb.org/download/{pdb_result[:4].upper()}.pdb",f"{pdb_database}/{pdb_result[:4].upper()}.pdb")
                        qualified_pdb= pdb_result.upper()
                        with open(f"./{protein_identifier}_psiblast_identified_x-ray_diffraction_structure.txt","a") as f:
                            f.write("{} {} \n".format(str(pdb_result.upper()),str(pdb_result_percent_identity)))
                        print("Run was successful")
                        break
                except:
                    print(":::Cannot build PDB for PDB ID, failed to download:" + pdb_result[:4])
                    os.system(f"echo '{pdb_result[:4]}' >> {open_awsem}/notExistPDBsList")
    else:
        with open(f"./{protein_identifier}_psiblast_identified_x-ray_diffraction_structure.txt","r") as f:
            for line in f:
                qualified_pdb = (line.strip("\n").split()[0])
                pdb_result_percent_identity=(line.strip("\n").split()[1])
    if not qualified_pdb:
        print(f"There are no qualifying crystal PDB structures for {protein_identifier}")
        return None
    else:
        print(f"The crystal structure of {protein_identifier} is: {qualified_pdb}")
        return qualified_pdb

In [5]:
#We set-up the necessary files to run the AWSEM simulation on Nots.
def AWSEM_calculations_setup(protein_identifier,open_awsem,parent_protein_folder,pdb_database,
                             fragment_memory,existing_structure,copy_ssweight):
    if not os.path.exists(f"./{protein_identifier}_openawsem_output.txt"):
        print("#####")
        print(f"Preparing OpenAWSEM Files for {protein_identifier}")
        print("#####")  

        with open(f"./{protein_identifier}_openawsem_output.txt", "w") as output:
            commands=["python",f"{open_awsem}/mm_create_project.py"]
            if existing_structure:
                commands.extend([f"{pdb_database}/{protein_identifier[:4]}.pdb", "--extended"])
                if protein_identifier[4:]:
                    commands.extend(["--c", f"{protein_identifier[4:]}"])
            else:
                commands.extend([f"./{protein_identifier}.fasta", "--predict_ssweight_from_fasta"])
            if fragment_memory:
                commands.append("--frag")
            print("The commands are: "); print(commands)
            subprocess.call(commands,stdout=output)
            #In order to ensure that the difference in AWSEM energy is due to changes in the contact energy, 
            #we copy the ssweight and ssweight.stride files below from the parent folder (these can change the AWSEM rama term)
            if copy_ssweight:
                copyfile(f"{parent_protein_folder}/ssweight.stride","./ssweight.stride")
                copyfile(f"{parent_protein_folder}/ssweight","./ssweight")

In [6]:
#Here we align the parent protein crystal structure sequence to the family MSA and mutate it in regions of 
#difference betweent aligned parent ENSEMBL sequence and the pseudogene sequence.
def align_parentprotein_structure_sequence(protein_family, protein_file_name, hmm_files,family_hmm_files):
    if not os.path.exists(f"./{protein_file_name}_aligned.sto"):
        print("#####")
        print(f"Aligning {protein_file_name} to {protein_family} Family MSA")
        print("#####")
        #The fasta file is created during the AWSEM energy calculations by "mm_create_project.py"
        cmd= [f"{hmm_files}/hmmalign {family_hmm_files}/{protein_family}.hmm ./{protein_file_name[:4]}.fasta"]
        with open(f"./{protein_file_name}_aligned.sto", "wb", 0) as output_file:
            subprocess.check_call(cmd, stdout=output_file,shell=True)

In [7]:
#The function below transform files from the stockholm to fasta format.
def pseudofam_convert_stockholm_to_fasta(input_file_name, output_file_name=None):
    from Bio import AlignIO

    if output_file_name == None:
        output_file_name = input_file_name.rstrip(".sto") + ".fasta"

    input_handle = open(input_file_name, "rU")
    output_handle = open(output_file_name, "w")

    alignments = AlignIO.parse(input_handle, "stockholm")
    AlignIO.write(alignments, output_handle, "fasta")

    output_handle.close()
    input_handle.close()

In [8]:
def pseudofam_get_sublist_complement(list_name, indices_list):
    return [list_name[i] for i in range(len(list_name)) if i not in indices_list]

In [9]:
def pseudofam_filter_inserts_and_gaps(input_file_name, gap_threshold=0.25):
    from Bio import AlignIO
    # rewrite Stockholm alignment in FASTA format
    input_handle = open(input_file_name, "rU")
    alignment = AlignIO.read(input_handle, "fasta")
    input_handle.close()

    output_handle = open(input_file_name.rstrip(".fasta")+"_gaps_and_inserts_filtered_up.fasta", "w")

    index_mask = []
    for i, record in enumerate(alignment): #HJ: This associates a number to each element
        index_mask += [i for i, x in enumerate(list(record.seq)) if x != x.upper()] #HJ: This logs each number 
        #(an index essentially) to each element in a string (here the sequence of a gene in the alignment) if 
        #the element is not equal to its uppercase (so essentially any lowercase letter, which represent any 
        #unaligned letters. Gaps meet this condition). This mask should not remove aligned sequences.
        
        #I define indexes of both the preserved and removed residues in the aligned sequence.
        #This necessary to "create" a pseudogene sequence from the parent pdb sequence with the 
        #subsitutions found when comapring the two aligned sequences.
        sequence=[x for i, x in enumerate(list(record.seq)) if x != "-"]
        print("The sequence without dashes is ", sequence)
        unaligned_mask=[i for i, x in enumerate(list(sequence)) if x != x.upper()]
        removed_residues=[x for i, x in enumerate(list(sequence)) if x != x.upper()]
        aligned_mask=[i for i, x in enumerate(list(sequence)) if x == x.upper()]
    index_mask = sorted(list(set(index_mask)));aligned_mask=sorted(list(set(aligned_mask)))
    unaligned_mask=sorted(list(set(unaligned_mask)))
    sequences_passed_threshold = 0
    for i, record in enumerate(alignment):
        aligned_sequence = pseudofam_get_sublist_complement(list(record.seq), index_mask)
        if float(aligned_sequence.count("-"))/len(aligned_sequence) < gap_threshold:
            output_handle.write(">%s\n" % record.id + "".join(aligned_sequence)+"\n")
            sequences_passed_threshold += 1
    print("%d sequences passed threshold." % sequences_passed_threshold)
    output_handle.close()
    return removed_residues, unaligned_mask, aligned_mask

In [10]:
#This function seeks to find the substitutions (ignoring any dashes) between the pseudogene and the parent 
#structure sequence.This function then builds a pseudogene structure sequence that is the parent structure 
#sequence featuring these subsitutions. 
def mutate_and_map_pdb_sequence(protein_file_name,pseudogene_identifier,protein_family,removed_residues,
                                unaligned_mask,aligned_mask,aligned_protein_repository):
    #Here we extract the aligned pseudogene sequence and the aligned parent structure sequence
    with open(f"{aligned_protein_repository}/{protein_family}_pseudogenes_aligned_gaps_and_inserts_filtered_up.fasta","r") as f:
        for line in f:
            if pseudogene_identifier in line:
                aligned_protein_sequence=next(f)
                print("The aligned pseudogene sequence is: {} \n".format(aligned_protein_sequence))
    with open(f"./{protein_file_name}_aligned_gaps_and_inserts_filtered_up.fasta","r") as f:
        for line in f:
            if ">" in line:
                aligned_parent_structure_sequence=next(f)
                print("The aligned parent sequence is: {} \n".format(aligned_parent_structure_sequence))

    print("The removed residues for the parent are: {} \n".format(removed_residues))
    print("The indices of the removed residues for the parent are: {} \n".format(unaligned_mask))
    print("The indices of the remaining residues for the parent are: {} \n".format(aligned_mask))
    substitutions_index=[i for i in range(len(aligned_parent_structure_sequence)) if \
                         aligned_parent_structure_sequence[i] != aligned_protein_sequence[i] and \
                         aligned_parent_structure_sequence[i] != "-" and aligned_protein_sequence[i] != "-"]
    print("The substitution indices are: {} \n".format(substitutions_index))
    #Here we mutate the parent structure sequence based on substitutions identified from comparing 
    #the aligned sequences (indices determined immediately above)
    aligned_parent_structure_sequence=list(aligned_parent_structure_sequence)
    for i in substitutions_index:
        aligned_parent_structure_sequence[i]=aligned_protein_sequence[i]
    aligned_parent_structure_sequence_with_gaps="".join(aligned_parent_structure_sequence)
    print("The mutated, aligned parent/homolog sequence is (with gaps): {} \n".format(aligned_parent_structure_sequence_with_gaps))
    aligned_parent_structure_sequence_without_gaps=aligned_parent_structure_sequence_with_gaps.replace("-","")
    print("The mutated, aligned parent/homolog sequence is (without gaps): {} \n".format(aligned_parent_structure_sequence_without_gaps))
        
    #Here we "build" the pseudogene structure sequence.
    mutated_structure_sequence=[i for i in range(len(unaligned_mask)+len(aligned_mask))]
    for (i, j) in zip(unaligned_mask,removed_residues):
        mutated_structure_sequence[i]=j.upper()
    for (i, j) in zip(aligned_mask,aligned_parent_structure_sequence_without_gaps):
        mutated_structure_sequence[i]=j      
    print("The pseudogene sequence is: {} \n".format("".join(mutated_structure_sequence)))
    
    return "".join(mutated_structure_sequence),len(substitutions_index)

In [11]:
def protein_pairs_AWSEM_setup(parent_identifier,protein_family,fragment_memory,
                              open_awsem,hmm_files,family_hmm_files,psiblast_repository,family_repository,
                              aligned_pseudogene_repository,aligned_parentprotein_repository):
    #We begin by setting up for the pair's parent protein AWSEM calculation.
    try:
        parent_protein_folder=os.path.join(family_repository,parent_identifier)
        os.mkdir(parent_protein_folder)            
    except OSError:
        pass
    os.chdir(parent_protein_folder)
    #Here we find the homologs for the parent sequence after extracting it below. As these are sequences I haven't
    #analyzed before, I need to retrieve them from Ensembl.
    server = f"https://rest.ensembl.org/sequence/id/{parent_identifier}?"
    r = requests.get(server, headers={ "Content-Type" : "text/plain"})
    parent_sequence=r.text
    
    #We check to see if the parent protein has a complete crystal structure in PDB.
    psiblast_PDB_results=blast_pdb(parent_identifier,parent_sequence,open_awsem,psiblast_repository) 
    qualified_PDB_id=find_best_pdb_for_sequence(parent_identifier,psiblast_PDB_results,os.path.join(open_awsem,"PDBs"),open_awsem)
    if qualified_PDB_id:
        with open("../../protein_family_parent_protein_crystal_structures.txt","a") as f:
            f.write("{} {} {} \n".format(protein_family,parent_identifier, qualified_PDB_id))
    #     if qualified_PDB_id:
    #         #Here we modify the PDB file to include only the relevant chains and remove any heteratoms.
    #         file_name=qualified_PDB_id; existing_structure=True
    #     else:
    #         file_name=f"{parent_identifier}_sequence"; existing_structure=False

    #     AWSEM_calculations_setup(file_name,open_awsem,os.getcwd(),os.path.join(open_awsem,"PDBs"),
    #                              fragment_memory,existing_structure,copy_ssweight=False)                                                                                               
    #     #The following lines of code align the parent protein sequence to the family MSA and identify substitution 
    #     #sites between the pseudogene and aligned parent sequence (mapping it to the full parent protein sequence).
    #     align_parentprotein_structure_sequence(protein_family, file_name, hmm_files,family_hmm_files)
    #     pseudofam_convert_stockholm_to_fasta(f"./{file_name}_aligned.sto")            
    #     removed_residues,removed_residue_indices,remaining_residue_indices = pseudofam_filter_inserts_and_gaps(f"./{file_name}_aligned.fasta",
    #                                                                                                              gap_threshold=.25)
    #     pseudogene_sequence, parent_protein_pseudogene_substitution_count= mutate_and_map_pdb_sequence(file_name,
    #                                                                                                    pseudogene_identifier,
    #                                                                                                    protein_family,
    #                                                                                                    removed_residues,
    #                                                                                                    removed_residue_indices,
    #                                                                                                    remaining_residue_indices,
    #                                                                                                    aligned_pseudogene_repository)


    #     ###
    #     #Now we begin setting up for the pair's pseudogene AWSEM calculation.
    #     try:
    #         pseudogene_folder=os.path.join(family_repository,pseudogene_identifier)
    #         os.mkdir(pseudogene_folder)            
    #     except OSError:
    #         pass
    #     os.chdir(pseudogene_folder)
    #     with open(f"./{pseudogene_identifier}.fasta","w") as f:
    #         f.write(">{}\n{}".format(pseudogene_identifier,pseudogene_sequence))
    #     AWSEM_calculations_setup(pseudogene_identifier,open_awsem,parent_protein_folder,os.path.join(open_awsem,"PDBs"),
    #                              fragment_memory,existing_structure=False,copy_ssweight=True)

    #     ###
    #     with open(f"{family_repository}/{protein_family}_parentprotein_pseudogene_pair_substitutions_AWSEM_energies.txt", "a") as f:
    #         f.write("{} {} {} \n".format(parent_identifier,pseudogene_identifier,str(parent_protein_pseudogene_substitution_count)))


In [12]:
def family_protein_pair_AWSEM_setup(protein_family,parent_protein,fragment_memory,
                                    AWSEM_results_directory=families_analysis_repository,
                                    open_awsem=openawsem_repository,hmm_files=hmm_script_repository,
                                    family_hmm_files=family_hmm_repository,psiblast_repository=ncbi_blast_repository,
                                    aligned_pseudogene_repository=family_pseudogene_aligned_files_repository,
                                    aligned_parentprotein_repository=family_parentprotein_aligned_files_repository):
    print("#####")
    print(f"Beginning Setup for AWSEM Calculations of {protein_family} Pseudogene-Parent Protein Pair")
    print("#####")      
    
    #We first create a repository for the selected protein family to store all of the files produced by this program,
    print(AWSEM_results_directory)
    protein_family_repository=os.path.join(AWSEM_results_directory,protein_family)
    try:
        os.mkdir(protein_family_repository)
    except OSError:
        pass
    os.chdir(protein_family_repository)
    
    protein_pairs_AWSEM_setup(parent_protein,protein_family,fragment_memory,
                              open_awsem,hmm_files,family_hmm_files,psiblast_repository,protein_family_repository,
                              aligned_pseudogene_repository,aligned_parentprotein_repository)

### Protein families (listed below) are extracted from the results Michael found from the PsiDR data. These are families with at least more than one pseudogene sharing the same parent protein with indications that they can be translated.

In [13]:
qualified_protein_families={"PF05649":"ENSP00000302051","PF03719":"ENSP00000341885","PF01431":"ENSP00000302051",
                            "PF00333":"ENSP00000341885"}
os.chdir(families_analysis_repository)

for i in qualified_protein_families:
    family_protein_pair_AWSEM_setup(i,qualified_protein_families[i],False)
            

#####
Beginning Setup for AWSEM Calculations of PF05649 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000302051 to Find Its Complete Structures
#####
The protein sequence is MEPPYSLTAHYDEFQEVKYVSRCGAGGARGASLPPGFPLGAARSATGARSGLPRWNRREVCLLSGLVFAAGLCAILAAMLALKYLGPVAAGGGACPEGCPERKAFARAARFLAANLDASIDPCQDFYSFACGGWLRRHAIPDDKLTYGTIAAIGEQNEERLRRLLARPGGGPGGAAQRKVRAFFRSCLDMREIERLGPRPMLEVIEDCGGWDLGGAEERPGVAARWDLNRLLYKAQGVYSAAALFSLTVSLDDRNSSRYVIRIDQDGLTLPERTLYLAQDEDSEKILAAYRVFMERVLSLLGADAVEQKAQEILQVEQQLANITVSEHDDLRRDVSSMYNKVTLGQLQKITPHLRWKWLLDQIFQEDFSEEEEVVLLATDYMQQVSQLIRSTPHRVLHNYLVWRVVVVLSEHLSPPFREALHELAQEMEGSDKPQELARVCLGQANRHFGMALGALFVHEHFSAASKAKVQQLVEDIKYILGQRLEELDWMDAETRAAARAKLQYMMVMVGYPDFLLKPDAVDKEYEFEVHEKTYFKNILNSIRFSIQLSVKKIRQEVDKSTWLLPPQALNAYYLPNKNQMVFPAGILQPTLYDPDFPQSLNYGGIGTIIGHELTHGYDDWGGQYDRSGNLLHWWTEAS

There are no qualifying crystal PDB structures for ENSP00000341885
#####
Beginning Setup for AWSEM Calculations of PF01431 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000302051 to Find Its Complete Structures
#####
The protein sequence is MEPPYSLTAHYDEFQEVKYVSRCGAGGARGASLPPGFPLGAARSATGARSGLPRWNRREVCLLSGLVFAAGLCAILAAMLALKYLGPVAAGGGACPEGCPERKAFARAARFLAANLDASIDPCQDFYSFACGGWLRRHAIPDDKLTYGTIAAIGEQNEERLRRLLARPGGGPGGAAQRKVRAFFRSCLDMREIERLGPRPMLEVIEDCGGWDLGGAEERPGVAARWDLNRLLYKAQGVYSAAALFSLTVSLDDRNSSRYVIRIDQDGLTLPERTLYLAQDEDSEKILAAYRVFMERVLSLLGADAVEQKAQEILQVEQQLANITVSEHDDLRRDVSSMYNKVTLGQLQKITPHLRWKWLLDQIFQEDFSEEEEVVLLATDYMQQVSQLIRSTPHRVLHNYLVWRVVVVLSEHLSPPFREALHELAQEMEGSDKPQELARVCLGQANRHFGMALGALFVHEHFSAASKAKVQQLVEDIKYILGQRLEELDWMDAETRAAARAKLQYMMVMVGYPDFLLKPDAVDKEYEFEVHEKTYFKNILNSIRFSIQLSVKKIRQEVDKSTWLLPPQALNA

There are no qualifying crystal PDB structures for ENSP00000341885


### As shown above, no parent proteins have a complete crystal structure. I repeat the process again but this time try to find any protein families with a parent protein with a complete crystal structure.

In [14]:
qualified_protein_families={'PF15433': 'ENSP00000315397', 'PF00578': 'ENSP00000298510', 'PF10417': 'ENSP00000298510', 'PF00478': 'ENSP00000321584', 'PF00571': 'ENSP00000321584', 'PF13639': 'ENSP00000426503', 'PF00160': 'ENSP00000361918', 'PF00956': 'ENSP00000417919', 'PF01431': 'ENSP00000302051', 'PF05649': 'ENSP00000302051', 'PF00501': 'ENSP00000350012', 'PF00400': 'ENSP00000359149', 'PF05191': 'ENSP00000371230', 'PF00406': 'ENSP00000371230', 'PF03953': 'ENSP00000326042', 'PF00091': 'ENSP00000326042', 'PF07686': 'ENSP00000312158', 'PF11627': 'ENSP00000333504', 'PF00076': 'ENSP00000350877', 'PF00333': 'ENSP00000341885', 'PF03719': 'ENSP00000341885', 'PF03465': 'ENSP00000353741', 'PF03463': 'ENSP00000353741', 'PF03464': 'ENSP00000353741', 'PF10409': 'ENSP00000361021', 'PF00782': 'ENSP00000380530', 'PF00581': 'ENSP00000380530', 'PF00038': 'ENSP00000293308', 'PF16208': 'ENSP00000293308', 'PF01652': 'ENSP00000389624', 'PF11717': 'ENSP00000312244', 'PF05712': 'ENSP00000312244', 'PF02023': 'ENSP00000355053', 'PF13405': 'ENSP00000362807', 'PF13499': 'ENSP00000362807', 'PF00935': 'ENSP00000404375', 'PF03946': 'ENSP00000354739', 'PF00298': 'ENSP00000354739', 'PF14904': 'ENSP00000432790', 'PF17832': 'ENSP00000360365', 'PF01472': 'ENSP00000360365', 'PF00198': 'ENSP00000335304', 'PF00364': 'ENSP00000335304', 'PF01246': 'ENSP00000260443', 'PF12937': 'ENSP00000359149', 'PF07654': 'ENSP00000351621', 'PF00012': 'ENSP00000432884', 'PF08075': 'ENSP00000409773', 'PF12934': 'ENSP00000418823', 'PF12933': 'ENSP00000418823', 'PF03066': 'ENSP00000377408', 'PF00248': 'ENSP00000235835', 'PF00179': 'ENSP00000429419', 'PF00118': 'ENSP00000275603', 'PF13837': 'ENSP00000355053', 'PF00096': 'ENSP00000355053', 'PF00210': 'ENSP00000273550', 'PF03143': 'ENSP00000339063', 'PF03144': 'ENSP00000339063', 'PF00009': 'ENSP00000339063', 'PF01187': 'ENSP00000385714', 'PF01459': 'ENSP00000265333', 'PF01015': 'ENSP00000424547', 'PF01283': 'ENSP00000348849', 'PF00615': 'ENSP00000426255', 'PF00787': 'ENSP00000426255', 'PF08628': 'ENSP00000426255', 'PF02194': 'ENSP00000426255', 'PF01158': 'ENSP00000378081', 'PF01248': 'ENSP00000230050', 'PF00723': 'ENSP00000362646', 'PF03357': 'ENSP00000223500', 'PF13765': 'ENSP00000312158', 'PF00622': 'ENSP00000312158', 'PF10559': 'ENSP00000243253', 'PF00344': 'ENSP00000243253', 'PF01199': 'ENSP00000378163', 'PF06229': 'ENSP00000226798', 'PF00071': 'ENSP00000387286'}
os.chdir(families_analysis_repository)

for i in qualified_protein_families:
    family_protein_pair_AWSEM_setup(i,qualified_protein_families[i],False)

#####
Beginning Setup for AWSEM Calculations of PF15433 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000315397 to Find Its Complete Structures
#####
The protein sequence is MFPRVSTFLPLRPLSRHPLSSGSPETSAAAIMLLTVRHGTVRYRSSALLARTKNNIQRYFGTNSVICSKKDKQSVRTEETSKETSESQDSEKENTKKDLLGIIKGMKVELSTVNVRTTKPPKRRPLKSLEATLGRLRRATEYAPKKRIEPLSPELVAAASAVADSLPFDKQTTKSELLSQLQQHEEESRAQRDAKRPKISFSNIISDMKVARSATARVRSRPELRIQFDEGYDNYPGQEKTDDLKKRKNIFTGKRLNIFDMMAVTKEAPETDTSPSLWDVEFAKQLATVNEQPLQNGFEELIQWTKEGKLWEFPINNEAGFDDDGSEFHEHIFLEKHLESFPKQGPIRHFMELVTCGLSKNPYLSVKQKVEHIEWFRNYFNEKKDILKESNIQFN
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/op

#####
Blasting ENSP00000298510 to Find Its Complete Structures
#####
The protein sequence is MAAAVGRLLRASVARHVSAIPWGISATAALRPAACGRTSLTNLLCSGSSQAKLFSTSSSCHAPAVTQHAPYFKGTAVVNGEFKDLSLDDFKGKYLVLFFYPLDFTFVCPTEIVAFSDKANEFHDVNCEVVAVSVDSHFSHLAWINTPRKNGGLGHMNIALLSDLTKQISRDYGVLLEGSGLALRGLFIIDPNGVIKHLSVNDLPVGRSVEETLRLVKAFQYVETHGEVCPANWTPDSPTIKPSPAASKEYFQKVNQ
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000298510_sequence.fasta
homologs: ['5jcg_I', '201', '408', '1048', '1.18e-145', '100.000', '76']
homologs: ['5jcg_H', '201', '408', '1048', '1.18e-145', '100.000', '76']
homologs: ['5jcg_G', '201', '408', '1048', '1.18e-145', '100.000', '76']
homologs: ['5jcg_F', '201', '408', '1048', '1.18e-145', '100.000', '76']
homologs: ['5jcg_E', '201', '408', '1048', '1.18e-145', '100.

#####
Blasting ENSP00000321584 to Find Its Complete Structures
#####
The protein sequence is MADYLISGGTSYVPDDGLTAQQLFNCGDGLTYNDFLILPGYIDFTADQVDLTSALTKKITLKTPLVSSPMDTVTEAGMAIAMALTGGIGFIHHNCTPEFQANEVRKVKKYEQGFITDPVVLSPKDRVRDVFEAKARHGFCGIPITDTGRMGSRLVGIISSRDIDFLKEEEHDCFLEEIMTKREDLVVAPAGITLKEANEILQRSKKGKLPIVNEDDELVAIIARTDLKKNRDYPLASKDAKKQLLCGAAIGTHEDDKYRLDLLAQAGVDVVVLDSSQGNSIFQINMIKYIKDKYPNLQVIGGNVVTAAQAKNLIDAGVDALRVGMGSGSICITQEVLACGRPQATAVYKVSEYARRFGVPVIADGGIQNVGHIAKALALGASTVMMGSLLAATTEAPGEYFFSDGIRLKKYRGMGSLDAMDKHLSSQNRYFSEADKIKVAQGVSGAVQDKGSIHKFVPYLIAGIQHSCQDIGAKSLTQVRAMMYSGELKFEKRTSSAQVEGGVHSLHSYEKRLF
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000321584_sequence.fasta
homologs: ['6udq_H', '519', '1056', '2730', '0.0', '100.000', '100']
homologs: ['6udq_D', '519'

Run was successful
The crystal structure of ENSP00000321584 is: 1NFB_B
#####
Beginning Setup for AWSEM Calculations of PF00571 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000321584 to Find Its Complete Structures
#####
The protein sequence is MADYLISGGTSYVPDDGLTAQQLFNCGDGLTYNDFLILPGYIDFTADQVDLTSALTKKITLKTPLVSSPMDTVTEAGMAIAMALTGGIGFIHHNCTPEFQANEVRKVKKYEQGFITDPVVLSPKDRVRDVFEAKARHGFCGIPITDTGRMGSRLVGIISSRDIDFLKEEEHDCFLEEIMTKREDLVVAPAGITLKEANEILQRSKKGKLPIVNEDDELVAIIARTDLKKNRDYPLASKDAKKQLLCGAAIGTHEDDKYRLDLLAQAGVDVVVLDSSQGNSIFQINMIKYIKDKYPNLQVIGGNVVTAAQAKNLIDAGVDALRVGMGSGSICITQEVLACGRPQATAVYKVSEYARRFGVPVIADGGIQNVGHIAKALALGASTVMMGSLLAATTEAPGEYFFSDGIRLKKYRGMGSLDAMDKHLSSQNRYFSEADKIKVAQGVSGAVQDKGSIHKFVPYLIAGIQHSCQDIGAKSLTQVRAMMYSGELKFEKRTSSAQVEGGVHSLHSYEKRLF
finding homologs
executing::: /Users/hanajaafari/Desk

Run was successful
The crystal structure of ENSP00000321584 is: 1NFB_B
#####
Beginning Setup for AWSEM Calculations of PF13639 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000426503 to Find Its Complete Structures
#####
The protein sequence is MSTRKRRGGAINSRQAQKRTREATSTPEISLEAEPIELVETAGDEIVDLTCESLEPVVVDLTHNDSVVIVDERRRPRRNARRLPQDHADSCVVSSDDEELSRDRDVYVTTHTPRNARDEGATGLRPSGTVSCPICMDGYSEIVQNGRLIVSTECGHVFCSQCLRDSLKNANTCPTCRKKINHKRYHPIYI
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000426503_sequence.fasta
homologs: ['4ppe_B', '76', '154', '389', '2.77e-48', '97.260', '38']
homologs: [

...

There are no qualifying crystal PDB structures for ENSP00000378081
#####
Beginning Setup for AWSEM Calculations of PF01248 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000230050 to Find Its Complete Structures
#####
The protein sequence is MAEEGIAAGGVMDVNTALQEVLKTALIHDGLARGIREAAKALDKRQAHLCVLASNCDEPMYVKLVEALCAEHQINLIKVDDNKKLGEWVGLCKIDREGKPRKVVGCSCVVVKDYGKESQAKDVIEEYFKCKK
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000230050_sequence.fasta
homologs: ['6yam_O', '132', '265', '678', '1.91e-92', '100.000', '100']
homologs: ['6yal_O', '132', '265', '678', '1.91e-92', '100.000', '100'

Run was successful
The crystal structure of ENSP00000230050 is: 4KZZ_M
#####
Beginning Setup for AWSEM Calculations of PF00723 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000362646 to Find Its Complete Structures
#####
The protein sequence is MRSRSNSGVRLDGYARLVQQTILCHQNPVTGLLPASYDQKDAWVRDNVYSILAVWGLGLAYRKNADRDEDKAKAYELEQSVVKLMRGLLHCMIRQVDKVESFKYSQSTKDSLHAKYNTKTCATVVGDDQWGHLQLDATSVYLLFLAQMTASGLHIIHSLDEVNFIQNLVFYIEAAYKTADFGIWERGDKTNQGISELNASSVGMAKAALEALDELDLFGVKGGPQSVIHVLADEVQHCQSILNSLLPRASTSKEVDASLLSVVSFPAFAVEDSQLVELTKQEIITKLQGRYGCCRFLRDGYKTPKEDPNRLYYEPAELKLFENIECEWPLFWTYFILDGVFSGNAEQVQEYKEALEAVLIKGKNGVPLLPELYSVPPDRVDEEYQNPHTVDRVPMGKLPHMWGQSLYILGSLMAEGFLAPGEIDPLNRRFSTVPKPDVVVQVSILAETEEIKTILKDKGIYVETIAEVYPIRVQPARILSHIYSSLGCNNRMKLSGRPYRHMGVLGTSKLYDIRKTIFTFTPQFIDQQQFYLALDNKMIVEMLRTDLSYLCSRWRMTGQPTITFPISH

#####
Blasting ENSP00000312158 to Find Its Complete Structures
#####
The protein sequence is MESAAALHFSRPASLLLLLLSLCALVSAQFIVVGPTDPILATVGENTTLRCHLSPEKNAEDMEVRWFRSQFSPAVFVYKGGRERTEEQMEEYRGRTTFVSKDISRGSVALVIHNITAQENGTYRCYFQEGRSYDEAILHLVVAGLGSKPLISMRGHEDGGIRLECISRGWYPKPLTVWRDPYGGVAPALKEVSMPDADGLFMVTTAVIIRDKSVRNMSCSINNTLLGQKKESVIFIPESFMPSVSPCAVALPIIVVILMIPIAVCIYWINKLQKEKKILSGEKEFERETREIALKELEKERVQKEEELQVKEKLQEELRWRRTFLHAVDVVLDPDTAHPDLFLSEDRRSVRRCPFRHLGESVPDNPERFDSQPCVLGRESFASGKHYWEVEVENVIEWTVGVCRDSVERKGEVLLIPQNGFWTLEMHKGQYRAVSSPDRILPLKESLCRVGVFLDYEAGDVSFYNMRDRSHIYTCPRSAFSVPVRPFFRLGCEDSPIFICPALTGANGVTVPEEGLTLHRVGTHQSL
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000312158_sequence.fasta
homologs: ['4hh8_A', '224', '218', '556', '5.12e-67', '50.704', '40']
homologs: ['

#####
Blasting ENSP00000243253 to Find Its Complete Structures
#####
The protein sequence is MAIKFLEVIKPFCVILPEIQKPERKIQFKEKVLWTAITLFIFLVCCQIPLFGIMSSDSADPFYWMRVILASNRGTLMELGISPIVTSGLIMQLLAGAKIIEVGDTPKDRALFNGAQKLFGMIITIGQSIVYVMTGMYGDPSEMGAGICLLITIQLFVAGLIVLLLDELLQKGYGLGSGISLFIATNICETIVWKAFSPTTVNTGRGMEFEGAIIALFHLLATRTDKVRALREAFYRQNLPNLMNLIATIFVFAVVIYFQGFRVDLPIKSARYRGQYNTYPIKLFYTSNIPIILQSALVSNLYVISQMLSARFSGNLLVSLLGTWSDTSSGGPARAYPVGGLCYYLSPPESFGSVLEDPVHAVVYIVFMLGSCAFFSKTWIEVSGSSAKDVAKQLKEQQMVMRGHRETSMVHELNRYIPTAAAFGGLCIGALSVLADFLGAIGSGTGILLAVTIIYQYFEIFVKEQSEVGSMGALLF
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000243253_sequence.fasta
homologs: ['4cg7_A', '476', '966', '2496', '0.0', '99.790', '100']
homologs: ['4cg6_A', '476', '966', '2496', '0.0', '99.790', '100']

There are no qualifying crystal PDB structures for ENSP00000378163
#####
Beginning Setup for AWSEM Calculations of PF06229 Pseudogene-Parent Protein Pair
#####
/Users/hanajaafari/Desktop/Time-Devolution-Calc/AWSEM_Energy_Calculations/06_24_2020_Global_AWSEM_Energy_Calculation_Search_Complete_Coverage_Translated_Pseudogene_Candidates
#####
Blasting ENSP00000226798 to Find Its Complete Structures
#####
The protein sequence is MAEYSYVKSTKLVLKGTKTKSKKKKSKDKKRKREEDEETQLDIVGIWWTVTNFGEISGTIAIEMDKGTYIHALDNGLFTLGAPHKEVDEGPSPPEQFTAVKLSDSRIALKSGYGKYLGINSDGLVVGRSDAIGPREQWEPVFQNGKMALLASNSCFIRCNEAGDIEAKSKTAGEEEMIKIRSCAERETKKKDDIPEEDKGNVKQCEINYVKKFQSFQDHKLKISKEDSKILKKARKDGFLHETLLDRRAKLKADRYCK
finding homologs
executing::: /Users/hanajaafari/Desktop/ncbi-blast-2.10.0+/bin/psiblast -num_iterations 1 -word_size 3 -evalue 0.005 -outfmt '6 sseqid slen bitscore score evalue pident qcovs' -matrix BLOSUM62 -db /Users/hanajaafari/Desktop/openawsem/pdbDB -query ./ENSP00000226798_sequence.fasta
homologs: ['2yug

## 14 protein families have a parent protein with a crystal structure. 

One of the families, PF00406, is the protein family of adenylate kinase, which monitors the levels of and converts between ATP, ADP, and AMP. It converts through its open lid, which it closes after binding ATP and AMP to push out water and increase efficacy of reaction. The parent protein, ENSP00000371230 (also associated with PF05191), is associated with the pseudogene transcript ENST00000432033. Below we retrieve the pseudogene sequence and analyze it with EMBOSS Transeq. 

In [15]:
server = "https://rest.ensembl.org/sequence/id/ENST00000432033?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'GAAGCACCTCTCTAGCGGAGACCTGCTCCAGGATCAACATGCTGCGGGGCACAGAAATTAATGTGTTAGCTAACGCTTTCATTGACCAAAGGAAACTCATCCGAGATGATGTCATGACTCAGCTGGCCCTATATGAGCTGAAAAATCTCACTTAGTATAGCTAGCTGTTGGGTGGTTTTCCATGGACACTTCCACAGGCAGAAGCCCTAGACTAAGCTTCTTAGACAGACAAAGTGATTAGCCCGAATGTGCCCTTTGAGGTCATTAAACAATGCCTTACTGCTCGCTAGATTCATCCTGCCAGTGGCTGAGTCTACAACATTGAATTCAACCCTCCCAAAACTGTGGGCACTGATGATCTGACTGGGGATCCTCTCATTCAGCATGAGGATGATAAACCAGAGACGGTTATCAAAAGACTAAAGGCTTATGAAGCCCAAACAAAGCCAGTCCTGGAATATTACCAAGCTTTTCTACAAACTAAAGTTCCACAAAGAAGCCAGAAAGCTGCCATTACTC'

The retrieved sequence above, when translated, shows many premature stop codons (represented by asterisks). The stop codons can be seen when clicking this cell.

EAPL\*RRPAPGSTCCGAQKLMC\*LTLSLTKGNSSEMMS\*LSWPYMS\*KISLSIASCWVVF
HGHFHRQKP\*TKLLRQTK\*LARMCPLRSLNNALLLARFILPVAESTTLNSTLPKLWALMI
\*LGILSFSMRMINQRRLSKD\*RLMKPKQSQSWNITKLFYKLKFHKEARKLPLL

Families PF00571 and PF00478 have the parent protein ENSP00000321584, with pseudogene transcript ENST00000440965. Like above we can analyze this pseudogene transcript:

In [16]:
server = "https://rest.ensembl.org/sequence/id/ENST00000440965?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'ATGAGGGACTACCTATCAGCAGCAGCACCGGCTACGTGGTCGAGGACGGGTTCACTGCGCTGAAGGGGTGGGTTGCCCCTCCACACCTGTGGGTTTTTCTCGTTAGGTGGAACGAGAGACTTGGAAAAGAAAGACACAGACGCAAAGTATAGAGAAAGAAAAAAGGGGGCCCAGGGGACCGGCGTTCAGCATACGGAGGATCCCGCCGGCCTCTTAGTTCCCTTAGTATTTATTGATCATTTTTAGGTGTTTCTCGGAGAGGGGGGTGTGGCAGGGTCATAGAATAATAGTGGAGAGAAGGTCAGCAGATAAACACGTGAACAAAGGTCTCTGCATCAAGAACAAGGTAAAGAATTAAGTGCTGTGCTTCAGATATGCATACACATAAACATCTCAATGACTTAAAGAGCAGTAGTGCTGCCCGCATGTCCCACCTCCAGCCCTAAGGCGGTTTTCCCCTATCTCAGCAGATGGAACATACAATCAGGTTTTACACCGAGACATTCCATTGCCCAGGGACTGGCAGGAGACAGATGCCTTCCTCTTCTCTCAACTGCAAACAGGCGTTCCTTCCTCTTTTACTAATCCTCCTCAGCACAGACCCTTTATGGGTGTCGGGCTGGGGGACGGTCAGGTCTTTCCCTTCCCATGAGGCCATATTTCAGACTATCACATGGGGAGAAACCTTGGACAATACCTGGCTTTCCTAGGCAGAGGTCCCTGCGGCCTTCCGCAGTGTTTGTGTCCCTGGGTACTTGAGATTAGGGAGTGGTGATGACTCTTAACGAGCACGCTGCCTTCAAGCATCTGTTTAACAAAGCACATCTTGCACAGCCCTTAATCCACTTAACCCTGAGTTGACATAGCACATGTTTCAGGGAGCACAGGGTTGGGGGTAAGGTTACAGATTAACAGCATCTCGAGGCAGAAGAATTTTTCTTAGTACAGAACAAAATGGAGTCTCCTATGTCTACTTCTTTCTACACAGACACAGTAACA

Like above the pseudogene, when translated, has many stop codons:
MRDYLSAAAPATWSRTGSLR\*RGGLPLHTCGFFSLGGTRDLEKKDTDAKYRERKKGAQGT
GVQHTEDPAGLLVPLVFIDHF\*VFLGEGGVAGS\*NNSGEKVSR\*TREQRSLHQEQGKELS
AVLQICIHINISMT\*RAVVLPACPTSSPKAVFPYLSRWNIQSGFTPRHSIAQGLAGDRCL
PLLSTANRRSFLFY\*SSSAQTLYGCRAGGRSGLSLPMRPYFRLSHGEKPWTIPGFPRQRS
LRPSAVFVSLGT\*D\*GVVMTLNEHAAFKHLFNKAHLAQPLIHLTLS\*HSTCFREHRVGGK
VTD\*QHLEAEEFFLVQNKMESPMSTSFYTDTVTI\*SLFLFPTPCSSSSPVTRDSPTTTS\*
FSQDS\*TS\*LMRWT\*PQP\*PNGHSEDAADLLPHGHCDRGRPGHRDGSDGRYWFHSPQLHP
RVPGQ\*GAEGQEV\*TGLYHTPRGAEPLAHCG\*CVGGQDASWLLWHPHH\*DGYHGQQAGGH
RHLPRHRLSC\*EGPHHPPQ\*GDDAKDQASGGSSSVRLKEANEILQLSKKGSCLSSMIAMS
WWPLSPAPR\*RTETTLWPPRIPMSSCWAGQLWVPMRMTNTTWTCSPR\*ASMS\*AWTRPKG
TRCIRSPWCITSNKSNPTSR\*LGGTW\*QQPRPTT\*LTLVWMGWAGAWTARLHLHQPGSDS
LQSAPGHCCVQGGQAYPELWCAHHSRWWHPDHGACGQGPGPRSLHSDDGLPAGRHHGGPR
RVLLLRRNAAQEVPGHGLTGCHGEEQQQPETILQRRG\*GEDHAGCLGLHPGQRVHSEVRA
LPHSGHPAWLPGYRGPQPVCPSVHDVLRGAQV\*EADHVSPDRRWHPWPALLREVAVLX

## Iterating over other parent proteins from list of 14

In [42]:
parents=[]
with open("../../protein_family_parent_protein_crystal_structures.txt","r") as f:
    for line in f:
        parent_identifier=line.split()[1]
        parents.append(parent_identifier)
parents=(set(parents))
parents.remove("ENSP00000321584");parents.remove("ENSP00000371230")
parents

{'ENSP00000230050',
 'ENSP00000235835',
 'ENSP00000265333',
 'ENSP00000273550',
 'ENSP00000353741',
 'ENSP00000389624',
 'ENSP00000429419',
 'ENSP00000432884'}

### Protein Family: PF01248 (Ribosomal S12/Ribsomal L30), Parent Protein: ENSP00000230050, Pseudogene Transcript: ENST00000451515

In [35]:
server = "https://rest.ensembl.org/sequence/id/ENST00000451515?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'CATGGCCGAGGAAGGCAGTGCTGCTGGAGGTGTAATGGACATTAATACTGTTTTACAGGAGGTGCTGAAGACCGCCCTCATCCATGATGGCCTAGCATATGAAATTTGCAAAGCTGCCAAAGCCTCAGACAAGTGCCAAGCCCATCTTTGTGTGCTGTGTGTGCTTGCATCCAACTGTGATGAGCCTATGTATGTCAAGTTGGTGGAGGCCCTTTGTGCTGAACACCAAATCAACCTAATTAAGGTTGATGACCAGAAACTAGGGGAATCGGTAGGCCTCTGTAAAACTGACAGAGAGGGGAAACCGTGTAAAGTGGTTGGTTGAAGTTGTATAGTAGTTACGAACTATGGCAAGGAGTCTCAGGCCAAGGATGTCATTGAAGAGTACTTCAAATGCAAGA'

Pseudogene Translated:
HGRGRQCCWRCNGH\*YCFTGGAEDRPHP\*WPSI\*NLQSCQSLRQVPSPSLCAVCACIQL\*
\*AYVCQVGGGPLC\*TPNQPN\*G\*\*PETRGIGRPL\*N\*QRGETV\*SGWLKLYSSYELWQGV
SGQGCH\*RVLQMQX

### Protein Family: PF00248 (Aldo/Keto Reductase), Parent Protein: ENSP00000235835, Pseudogene Transcript: ENST00000460134

In [36]:
server = "https://rest.ensembl.org/sequence/id/ENST00000460134?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'ATGTCCCGGCCCTCCACGGTGTTGGGTGCCATGGAGATGGGGCGCCGTAGTCACCCGCGCCTTCCTGGAGCACGGCCACACCAAGATAGACACGGCCTTCGTGTACAGCGACAGCCAGTCCAAGACCATCCTGGGTGGCCTGAAAATTGATACCAAGGCCAATCCATTGTTTGGGAACTCCCTGAAACCTGACAGTCTCCGGTTCCAGCTGGAGACGTCACTGAAGTGGCTGCAGTGTCCCCGAGTGGACCTCTTCTACCTGCTCAGGCCAGACCACAGCACCCCGGTGGAAGAGACACTGAGTGCCTGCCACCAGCTGCACCAGGAGGGCAAGTTCCTGGATCTTGGCCTCTCCAACTATGCCACCTGGGAAGTGGCCAAGATCTGTAACCTCTGCAAGAGCAATGGCTGGACCCTGCCCACTGTGTACCAGGGCATGTACAACGCCACCACCCAGCAGGTGAAAACGGAGCTCTTCCCCTGCCTCAGGCACTCTGGACTGAGGTTCTATGCCTTCAACCCTCTGGCTGGGGACCTGCTGACTGGCAAGTACAAGTATAAGGACAAGGACAGGAAACAGCCCATGGGCCGCTTCTTTGGGACTCAATGGGCAGAGATCTACAGGAATCGCTTCTGGAAGAAGCACCACTTCGATGGCATTGCCCTGGTAGAGAAGGCCCTGCAGGCCGCGTATGGCGCCAGCGCCCCCAGCATGACCTCGGCTGCCCTCTGGTGGATGTACCACCACTCACAGCTGCAGGGTGCCCACGGGGACGCAGTCATCCTGGGCATGTCCAGCTTGGAGCAGTTGGAGCAGAACTTGGCAGCGGCAGAGGAAGGGCCCCTGGAGCCAGATGTTGTGGAGGCCTTTAATCAAGCCTGGCAATTGGTTGCTCACGAATGTCCCAACTACTT'

Pseudogene Translated: MSRPSTVLGAMEMGRRSHPRLPGARPHQDRHGLRVQRQPVQDHPGWPEN\*YQGQSIVWEL
PET\*QSPVPAGDVTEVAAVSPSGPLLPAQARPQHPGGRDTECLPPAAPGGQVPGSWPLQL
CHLGSGQDL\*PLQEQWLDPAHCVPGHVQRHHPAGENGALPLPQALWTEVLCLQPSGWGPA
DWQVQV\*GQGQETAHGPLLWDSMGRDLQESLLEEAPLRWHCPGREGPAGRVWRQRPQHDL
GCPLVDVPPLTAAGCPRGRSHPGHVQLGAVGAELGSGRGRAPGARCCGGL\*SSLAIGCSR
MSQLL

### Protein Family: PF01459 (Eukaryotic Porin), Parent Protein: ENSP00000265333, Pseudogene Transcript: ENST00000524809

In [38]:
server = "https://rest.ensembl.org/sequence/id/ENST00000524809?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'ATGGCTGTGCCACCCAAGTATGCCAATCTTGGCAAATCTGCCAGGAATGTCTTCAACAAGGGCTACGGATATGGCCTAATAAACCTTTTGAAAACAAAATCTGAGAATGCATTGGAATTTGAAAGCTTAGGCTCAGCCAACACTGAAATCACCAAAGTGACAGGCAGTCATGGAGACCAAGTACAGATGGACTGAGTACGGCCTGATGTTTATGAAGTGAAACACACAACACACCAGGACTGAGATTACTGTGGAAGATCATCTTGCAAGTGGACTGAAGCTGACCCTCAATTCACCCTTGTCACCTAATACTGGGGGAAAAATGCTAAAATCAACACCAGGTACAAGCAGGAGCACATCAACTTGGGCTGTAACCTGGATTTTGACATCACTGGACCTTCCATCTGGGGCGCTCTGGTGCTGGGTTACAAGGGTTGACTGGCTGGCTACCAGATAAATTTTGAGACTGCAAAATCCTGAGTGACCCAGAGCAACTCTGCAGTTGGTTACAAGTCTGATGAATTCCAACTTCACACTAACGTAAATAATGGGACAGTTTGGTGGCTCCATTTACCAGAAGGTGAGTAAGAAGTTGGAGACCACTGTCAATTTCACCTGGACAGCAGGAAACAGTAACACTAATACGGTTTGTCTGTGTCCCCACCCAAATCTAATCTTGAATTGTAGCTCCCATAATTCCCACGTGTGTGGGACTGACCTGGTGGGAGATAATTGAATCATGGGGGCAGTTTCCCCATACTGTTCTCATGGTAGTGAATGTCTCACGGGATCTGATGGTTTTATAAGGGGAAACTCCTTTCACTTGGCTTTCATTTTCTCTCCCGCCTGCCACCATGTAAGATGTGGCTTTCGCCTTCCCCCACAATGGTGAGGCCTCTCCAGCCACGTGGAACTGTGTGAGCCCATTAAACCTTTTCCTTTATAAATTACCCAGTCCCAAGGTGTATCTTTACTAGCAGTGTGAAAACAGACTAATAC

Pseudogene Translated: MAVPPKYANLGKSARNVFNKGYGYGLINLLKTKSENALEFESLGSANTEITKVTGSHGDQ
VQMD\*VRPDVYEVKHTTHQD\*DYCGRSSCKWTEADPQFTLVT\*YWGKNAKINTRYKQEHI
NLGCNLDFDITGPSIWGALVLGYKG\*LAGYQINFETAKS\*VTQSNSAVGYKSDEFQLHTN
VNNGTVWWLHLPEGE\*EVGDHCQFHLDSRKQ\*H\*YGLSVSPPKSNLEL\*LP\*FPRVWD\*P
GGR\*LNHGGSFPILFSW\*\*MSHGI\*WFYKGKLLSLGFHFLSRLPPCKMWLSPSPTMVRPL
QPRGTV\*AH\*TFSFINYPVPRCIFTSSVKTD\*YKHSLWNTQVSDQP\*HLLLG\*VNNSSLT
GLEYTETLKPGIKMTLSALLDGKNNNASGYTLGLELKX

### Protein Family: PF00210 (Ferritin), Parent Protein: ENSP00000273550, Pseudogene Transcript: ENST00000437933

In [39]:
server = "https://rest.ensembl.org/sequence/id/ENST00000437933?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'ATGACGACTGCGCGTCCCCCTCGCAGGTGCGCCAGAACTACCGCCATCAACCGCCAGATCAACGTGGAGCTCTATGCCTCCTACATCTACCTGTCCATGTCTTACTACTTTGACCACAATGATGTGGCTTTGAAGAACTTTGCCAAATACTTTCTTCACCAATCTCATGAGGAGAGGGAACATGCCAAGAAACTGATGAAGCTGCAGAACCAATGAGGTGGCTGAATCTTCTTCCTTCGGGACTTCAAGAAACCAGACCATGATGACTGGGAGAGCAGGCTGAATGCGATGGAGCATGCATTACATTTGGAAAAACTTGGGAATCAGTCACTACTGGAACTGCACAAACTGGCCACCGACAAAATTACCCCCATTTGTGTGACTTCACTGAAACACATTACCTGAATGAGCAGGGGAAATCCATCAAGAATTGGGTGACCATGTGACCAACTTGCGCAAGATGAGGGCACCCGAATCTGGCTTGGCAGAATATTTCTCTGACAAGCACACCCCGGGGAGACAGTGATAATGA'

Pseudogene Translated: MTTARPPRRCARTTAINRQINVELYASYIYLSMSYYFDHNDVALKNFAKYFLHQSHEERE
HAKKLMKLQNQ\*GG\*IFFLRDFKKPDHDDWESRLNAMEHALHLEKLGNQSLLELHKLATD
KITPICVTSLKHIT\*MSRGNPSRIG\*PCDQLAQDEGTRIWLGRIFL\*QAHPGETVIMX

### Protein Family: PF03465 (eRF1 domain 3), PF03464 (eRF1 domain 2), PF03463 (eRF1 domain 1); Parent Protein: ENSP00000353741; Pseudogene Transcript: ENST00000478788

In [41]:
server = "https://rest.ensembl.org/sequence/id/ENST00000478788?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'ATGGTGGACGACCCCAGTGCTGTGGACAGGAACGTGGAGTTCTGGAAGATCAAGAAGCTCATTAAGAGCTTGGAGGCGGGCTGGGCGCCGTGGCTCACGCCTGTAATCCCAGCACTTTGAGGCCGAGGCGGGCAGATCACCTGAGGTCAGGAGTTCAAGACCTGCCTGGCCAATATGGTGAAACCCTGTCTCTACTAAAAATACAAAAATTAGCTGGGTGTGGTGGCACATGCCTGTAGTCCCAGCTACTTGGGAGGCTGAGGCAGAAGAATCTTGTGAACCCGGGAGGCTTATGTGGCAGTGAGGCAAGATCACGCCACTGTACTCCAGCCTGGGCGACAGAGCTGTCTCAAAAAAAAAAAAAAAAAAAGGAGCTTGGAGGCAGCCTGCAGCAATGGCACTAGCATGATATCACTGATCATTCCTCCCAAAGACCAGATTTCACCAGTGGCAAAAATGTTATCAGATGAGTTTGGAACTGCATCTAACATTAAGGCAGGAGTAAACCGCCATCCAGTCCTGGGAGCCATTACATCTATACAACAAAGGCTCAAACTTTATAACAAAGTACCTCCAGATGGTCTGGTTGTTTACTGTGGAACAACTGTAACAGAAGAAGGAAAGGAAAAGAAAGTCAACGTTGACTTTGAATCTTTCAAACCAATTAATACGTCATTGTATTGTGTGACAACAAATTCCATAAAGGGGCTCTTACAGCACTACTTTCAGATGACAGCAAGTTTGGATTCATTGTAATAGATGGTAGTGGTGCACTTTTTGGCACACTCCAAGGAAACACAAGAGAAGTCCTGCACAAATTCACTGTGGATCTCCCAAAGAAACACAGTAGAGGCGTTTTGCCCGTTTAAAAATGGAAAAGCGACATAACTATGTTCAGAAAGTAGCAGAGACTGCTGTGCAGCTATTTCTGGGGACAAAGTGAATGTGGCTGGTCTAGTTTTAGCTGGATCTGCTGACTTTGAACTAAGTCAATCTGAT

Pseudogene Translated: MVDDPSAVDRNVEFWKIKKLIKSLEAGWAPWLTPVIPAL\*GRGGQIT\*GQEFKTCLANMV
KPCLY\*KYKN\*LGVVAHACSPSYLGG\*GRRIL\*TREAYVAVRQDHATVLQPGRQSCLKKK
KKKRSLEAACSNGTSMISLIIPPKDQISPVAKMLSDEFGTASNIKAGVNRHPVLGAITSI
QQRLKLYNKVPPDGLVVYCGTTVTEEGKEKKVNVDFESFKPINTSLYCVTTNSIKGLLQH
YFQMTASLDSL\*\*MVVVHFLAHSKETQEKSCTNSLWISQRNTVEAFCPFKNGKAT\*LCSE
SSRDCCAAISGDKVNVAGLVLAGSADFELSQSDMFDQRLQSKVLKLVDISYGGENGFNQA
TELSTEVLSEVKFIQEKKSIERYFDEITQDTRKYCFGVEDTLKTLEKGAVEILMVYENLD
IMRYVLYCQGTEKEKIL\*LQSKKRINLISQTKRPDRNMSLSRACPCWNGLLTTIKNLELC
WK\*SQIKCKTGLGL\*KDLVELEVSCSTE\*ISREWNTKEETMNFLTLMTX

### Protein Family: PF01652 (Initiation Factor 4E), Parent Protein: ENSP00000389624, Pseudogene Transcript: ENST00000422311

In [43]:
server = "https://rest.ensembl.org/sequence/id/ENST00000422311?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'ACTGTCAAACCAGAAACCACCCCTACTCCTAATCCCCCTGTGAAGAGGAGAAAACAGAATCTAATCAGGAGGTTGCTAACCCAGAACACTATATTAAACATCCTCTACAGAACAGATGGGCACTCTGGTTTTTTTAAAAAAGATAAAAGCAAAACTTGACAAGCAAACCTGCCGCTGATCTCTAAGTTTGATCCTGTTGACTTTTGGGCTCTGTACAACCATATCCAGTTGTCTAGTAATTTAATGTCTGGCTGTGACTACTCACTTTTTAAGGATGGTGTTGAGCCCATGTGGGAAGATGAGAAAAACAAACAGGGAGGATGATGGCTAATTTACATTGAACAAACAGCAGAGATGAAGGGACCTCAATCACTTTTGGCTAGAGACTCTACTGTGTCTTATTGGAGAATCTTTTGATGACTCAGTGATGATATATGTGCAGCTGTTCTTAATGTTAGAGCTAAAGGTGATAAGATAGCAGTATGGACTACTGAATGTGAAAACAGAGAAGCTGTTACACATATAGAGAGGGTATACAAGGAAAGGTTAGGACTTCCTCCAAACATCGTGATTGGTTATCAGTCCCATGCAGACACAGCTACTAAAAACAGCTCCACCACTAAAAATAGGTTTGTTGTTTAA'

Pseudogene Translated: TVKPETTPTPNPPVKRRKQNLIRRLLTQNTILNILYRTDGHSGFFKKDKSKT\*QANLPLI
SKFDPVDFWALYNHIQLSSNLMSGCDYSLFKDGVEPMWEDEKNKQGG\*WLIYIEQTAEMK
GPQSLLARDSTVSYWRIF\*\*LSDDICAAVLNVRAKGDKIAVWTTECENREAVTHIERVYK
ERLGLPPNIVIGYQSHADTATKNSSTTKNRFVV\*

### Protein Family: PF00179 (Ubiquitin Conjugating Enzyme), Parent Protein: ENSP00000429419, Pseudogene Transcript: ENST00000433562

In [44]:
server = "https://rest.ensembl.org/sequence/id/ENST00000433562?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'CATGATGACATCACTCCAGATGGTATCTTCCAGTTCTCAGGTCAAAACCTTTGAGTTATGCTTATGCAGCTATTGATAGGCAATGGTATGATTAACTGGGGCCTTAAAGATGATGAAGATATGACATTTACAAGGTGGACAGGCATGATTATTGGGCCACCAAGGACAAATTATGAAAAGAGAATATAGGCCGGGCATGGCAGTTCACACCTATAATCCCAGAACTTTGGGAGGCCAAGGAGGGCAGATCACGAGGTCAGGAGTTTGAGACTAGTCTGCCCAACATAGTGAAACCTCATCTCTACTAAAAATACAAAAAATTAGCTGGGTGTGGTGGTGTGCACCTGTAATCCCAGCTACTCGGGAAGCTGAGGCAGGAGAATCACATGAACCTAGGAAGCAGAGGTTGCAGTGAGCCGAGATCACGCCACTGCGCTCCCGTGTGGGCCACAGCGCGAGCCTCTGTCTCAAAAATAAAAAAAAAGAATATATAGCCTGAAAATAGAAAGTGGACCCAAATACCCAGAAGCTCCCCCATCAGTTAGATTTGTAACAAGAATTAATATGAATGGAATAAATAACTCCAGTGGGATGGTGGATGACCGCAGCACACCAGTGTTAGCAAAATGACAAAATTCATATAGCATTAAAGTTGTACTTCAGGAGCTAAGACGTCTAATGATGTCCAAAGAAAATATGAAGCTTCCACAGCCACTGGAAGGACAAACATACAACA'

Pseudogene Translated: HDDITPDGIFQFSGQNL\*VMLMQLLIGNGMINWGLKDDEDMTFTRWTGMIIGPPRTNYEK
RI\*AGHGSSHL\*SQNFGRPRRADHEVRSLRLVCPT\*\*NLISTKNTKN\*LGVVVCTCNPSY
SGS\*GRRIT\*T\*EAEVAVSRDHATALPCGPQREPLSQK\*KKRIYSLKIESGPKYPEAPPS
VRFVTRINMNGINNSSGMVDDRSTPVLAK\*QNSYSIKVVLQELRRLMMSKENMKLPQPLE
GQTYNX

### Protein Family: PF00012 (HSP70-heat shock proteins), Parent Protein: ENSP00000432884; Pseudogene Transcript: ENST00000401676 (deleted?)

In [45]:
server = "https://rest.ensembl.org/sequence/id/ENST00000401676?"
r = requests.get(server, headers={ "Content-Type" : "text/plain"})
pseudogene_sequence=r.text
pseudogene_sequence

'GCAGTTGTCACTGATCTTGGCACCACCTATTCCTCTGTAGGTGTCTTCCAGCACAGGACAGTAGAGATCATCACCACTCGGCAAGAAAACCAAGCTATGTTGCTTTTAGCTGACACAGAAAGATTGATTAGTGATGGTTTAGGAATCAAATTCAGTGGACTCCATCAACACAGTTTTAGATAACAGAGGTGTAATTAGACACCGATTTGACAGTCCAACTGTCTGGTCAAACATGAAAAGCACTGGACTTTCCTGGTTGTCGACTGTGCACACAGGCTGAAGATCCAGAGGAGGAAAGGGAGGGATACTAAAAGCTTTATACAGTTGAGGTATTTCCTGTGGTTGTGTTAGCTATTGCTGTGTAACAAATTACCCCAAAACTTGGTGGACTGAAACACCACTTACTGGCTGGGCACGGTGGCTCACGCCTGTTACCCCAGCACTTTGGTAGGCCAAGGCAGGCAGGTCACATGAGGCCAGGAGTTCAAGACCAGCCTGGCCAACATGGTGAAACCCCATAAATTAGTCGGGCATGGTGGCATGCACCTGTAGTCCCAAGTAGCTGACTCTGGAGGCTGAGGCATGAGAATCACTTGAACCCAGGAGGCAGAGGTTGGACTGAGCTGAGACTGCACCACTGCACTCCAGCCTGGGTGACAGAGTGAGACTGTCTCAAAAAATAAAAAGAAAAAAGAAACACCACTTACTATTTAAGTTTTTGTGGGTCAGGAATCAGTGCAGCTTAGCTGGGTCCTCTGATTCTGTGTTTCTCACAAGGCTGCAGTCATCTCAGAGATTCACTTCCTAGTTCACTCGTGGTTATTGGCTGGATTTGGTTCCTCTCAGGCTATTAGATTGAGGCCTCAGTTCCTCATAAGCTGATGTCCAGAAGCTTCTCCATAGAGCATCTCACAACATGGCAGTGTGCTTCATCAGATGAACAAGTAGGGGAGTGATATGGTTTAGATCTGTATCCCTGCCCAAATCTTGTGTCGAATG

Pseudogene Translated: AVVTDLGTTYSSVGVFQHRTVEIITTRQENQAMLLLADTERLISDGLGIKFSGLHQHSFR
\*QRCN\*TPI\*QSNCLVKHEKHWTFLVVDCAHRLKIQRRKGRDTKSFIQLRYFLWLC\*LLL
CNKLPQNLVD\*NTTYWLGTVAHACYPSTLVGQGRQVT\*GQEFKTSLANMVKPHKLVGHGG
MHL\*SQVADSGG\*GMRIT\*TQEAEVGLS\*DCTTALQPG\*QSETVSKNKKKKETPLTI\*VF
VGQESVQLSWVL\*FCVSHKAAVISEIHFLVHSWLLAGFGSSQAIRLRPQFLIS\*CPEASP
\*SISQHGSVLHQMNK\*GSDMV\*ICIPAQILCRMVMPVLEVGPGGGVTGSSG\*ISHEWFGT
IPLVLFL\*Q\*VLVRLFKSV\*HLHLPLFALLRFCHVRYLLSLHLPL\*L\*VSGGFPRSQAHA
SIMLPVEQCRTMSQLGLFSL\*ITPSQVFLYSSERLD\*YREQERLSGREC\*\*NRSHSLLQP
NHRSDFCCALFVKNKLGPGTVAHAYNPSPLGG\*DGRVT\*GQEFKTSLVNIVIPPPISLKK
KKN\*KASYGQSWWLTPVIPALREAEVSGSPEVRSSRPAWLTW\*NPVSTKNTKISWAWWWT
PVIPVTWEAEAGKSLEPKGGRGCSEPRSAWMKQQNFVSRKKKASY\*TEPTLEGRG\*HKSE
HQEVGSLWVAS\*KSAYHIGFEKDGENSGNLLYDP\*PTRSSYYQLL\*WLPASGHQSWWNHY
\*SHCVLNCQQAKCRWSQLRRER\*CQI\*\*SDLLPGWHFLSVHPQYSRWNY\*SQVCN\*GYPP
E\*RVP\*QLSGQLFHG\*DQLQL\*KGHPLRTSRLPKPLLPLLNVLFILVSRPI\*D\*VPP\*RG
SHLYLCAHEQLEESKATPVPGTLHLWRKP\*AMSPRHITSPSRVALFISPRFKISYX